In [1]:
from __future__ import print_function

%matplotlib inline

import sys
import tensorflow as tf
from tensorflow.python.framework import ops
from tensorflow.contrib.framework.python.ops import variables

from collections import OrderedDict

import numpy as np
import scipy.misc
from input import *
from model import *
from train import *
import matplotlib.pyplot as plt
tf.reset_default_graph()

1.3.0


In [2]:
image_shape = (160,576)
iterator, filename = get_train_inputs(batch_size=100,
                                      repeat=True, 
                                      num_classes=2, 
                                      image_shape=image_shape)
print(iterator, filename)

<tensorflow.contrib.data.python.ops.dataset_ops.Iterator object at 0x11713d828> Tensor("training_data/input:0", shape=(?,), dtype=string)


In [3]:
_show = False
if _show:
    with tf.Session() as sess:
        sess.run(iterator.initializer, feed_dict={filename: ['data/kitti_segmentation.tfrecord']})
        next_element = iterator.get_next()
        i = 1
        while i < 5:
            i += 1
            print("*"*10)
            image, label = sess.run(next_element)
            new = scipy.misc.imresize(image,(160*2,576*2) )
            plt.imshow(image[:,:,:])
            plt.show()
            plt.imshow(new[:,:,:])
#             plt.imshow(label[:,:,0], cmap='jet', alpha=0.5)
            plt.show()
            print(new.shape, image.shape)

In [4]:
# record_iterator = tf.python_io.tf_record_iterator(path='data/kitti_segmentation.tfrecord')
# string_record = next(record_iterator)
# example = tf.train.Example()
# example.ParseFromString(string_record)

## Model

In [5]:
encoder = SlimModelEncoder(name="vgg_16", num_classes=2, is_training=True)
image, label = iterator.get_next()
assign_op, feed_dict, end_points = encoder.build(image=image, image_shape=image_shape)

In [6]:
end_points.keys()
gr = tf.get_default_graph()
gr.get_collection('table_initializer')

[]

In [7]:
print(end_points['vgg_16/conv3/conv3_3'], 
    end_points['vgg_16/conv4/conv4_3'], 
    end_points['vgg_16/conv5/conv5_3'],
    end_points['vgg_16/fc6'],
    end_points['vgg_16/fc7'],sep='\n')

Tensor("vgg_16/conv3/conv3_3/Relu:0", shape=(1, 40, 144, 256), dtype=float32)
Tensor("vgg_16/conv4/conv4_3/Relu:0", shape=(1, 20, 72, 512), dtype=float32)
Tensor("vgg_16/conv5/conv5_3/Relu:0", shape=(1, 10, 36, 512), dtype=float32)
Tensor("vgg_16/fc6/Relu:0", shape=(1, 5, 18, 4096), dtype=float32)
Tensor("vgg_16/fc7/Relu:0", shape=(1, 5, 18, 4096), dtype=float32)


In [8]:
decoder = FCNDecoder(end_points, nb_classes=2, scope='decoder')

In [9]:
tensors_to_connect = OrderedDict()
tensors_to_connect["vgg_16/fc7"] = (2,2)
tensors_to_connect['vgg_16/conv5/conv5_3'] = (2,2)
tensors_to_connect['vgg_16/conv4/conv4_3'] = (8,8)

In [10]:
net = decoder.build(tensors_to_connect)

In [11]:
assert tuple(gr.get_tensor_by_name('logit:0').get_shape().as_list()[1:3]) == image_shape

## Train

In [12]:
trainer = Trainer(nb_classes=2, optimizer=tf.train.AdamOptimizer, learning_rate=3e-3)

In [ ]:
trainer.build(predictions=net, labels=label, decoder_scope='decoder')

pred shape (92160, 2), label shape (92160, 2)


In [ ]:
trainer.train(iterator, assign_op=assign_op, feed_dict=feed_dict, filename=['data/kitti_segmentation.tfrecord'])

INFO:tensorflow:Starting Session.
INFO:tensorflow:Saving checkpoint to path /tmp/tf/model.ckpt
INFO:tensorflow:Starting Queues.
INFO:tensorflow:global_step/sec: 0
INFO:tensorflow:global step 1: loss = 0.6938 (6.307 sec/step)
INFO:tensorflow:global step 2: loss = 0.6926 (2.749 sec/step)
INFO:tensorflow:Recording summary at step 2.
INFO:tensorflow:global step 3: loss = 0.6918 (3.084 sec/step)
INFO:tensorflow:global step 4: loss = 0.6911 (2.128 sec/step)
INFO:tensorflow:global step 5: loss = 0.6906 (2.101 sec/step)
INFO:tensorflow:global step 6: loss = 0.6892 (2.031 sec/step)
INFO:tensorflow:global step 7: loss = 0.6889 (2.238 sec/step)
INFO:tensorflow:global step 8: loss = 0.6874 (2.583 sec/step)
INFO:tensorflow:global step 9: loss = 0.6863 (2.356 sec/step)
INFO:tensorflow:global step 10: loss = 0.6871 (2.670 sec/step)
INFO:tensorflow:global step 11: loss = 0.6851 (2.663 sec/step)
INFO:tensorflow:global step 12: loss = 0.6856 (2.176 sec/step)
INFO:tensorflow:global step 13: loss = 0.6835

## Predict

In [ ]:
TRAIN_DIR = "/tmp/tf"
with tf.Graph().as_default() as graph:
    with tf.Session() as sess:
        saver = tf.train.import_meta_graph(os.path.join(TRAIN_DIR,"model.ckpt-1000.meta"))
        saver.restore(sess, os.path.join(TRAIN_DIR,"model.ckpt-1000"))
        # predictions and labels
        logit = graph.get_tensor_by_name('decoder/Conv2d_transpose_2/Relu:0')
        input_tensor = graph.get_tensor_by_name('training_data/input:0')   
        print(input_tensor)
        # (1, 160, 576, 2) -> (160, 576, 2)
        logit = tf.squeeze(logit)
        pred = tf.argmax(logit, axis=2)
        #sess.run(iterator.initializer, feed_dict={input_tensor: filename})
        pred = sess.run(pred)
        #print(pred, pred.shape())     

In [ ]:
gr = tf.get_default_graph()

In [ ]:
gr.collections